In [2]:
from sklearn import svm
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, roc_auc_score, recall_score, precision_score
from keras import models
from keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.layers import Embedding
import keras
from tensorflow.keras.utils import to_categorical
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from nltk.sentiment.vader import SentimentIntensityAnalyzer
stopword = {"URL"}|set(stopwords.words("english"))|set(stopwords.words("spanish"))|set(stopwords.words("portuguese"))|set(stopwords.words("french"))|set(stopwords.words("german"))

For this project, all of linear, tree based, and NN models would be used to evaluate the text to classfy whether the text is complaint or not. Here, we'll set 1 as noncomplaint and 0 as complaint. The main target for us is to identify more noncomplaint texts without too many false positive in our final dataset. As some of models do not perform well in our data, they wil be removed from this report to limit the page size. Here, logistic regression, SVM, Decision Tree, Random Forest, Boosting, Bagging, DNN, RNN, LSTM models were mainly used in this project.

In [11]:
def clean(s):
    s=[x.strip().lower() for x in s]
    #s=[re.sub("http[^\s]+|www\[^\s]+","url",x) for x in s]
    s=[re.sub('[^a-z]',' ',x) for x in s]
    return(s)
def doc2vec(text,wordlist):
    worddic=dict(zip(wordlist,range(len(wordlist))))
    cleaned = np.zeros((len(text),len(wordlist)))
    i=0
    for i in range(len(text)):
        for word in text[i].split():
            if word in wordlist:
                cleaned[i,worddic[word]]+=1
    return(cleaned)

def create_model(layer=1,layer_activation="relu",dense=16,optimizer='rmsprop'):
    model = models.Sequential()
    model.add(layers.Dense(dense,activation=layer_activation, input_shape=(5260,)))
    if layer>1:
        for i in range(layer-1):
            model.add(layers.Dense(dense, activation=layer_activation))
    model.add(layers.Dense(2,activation='sigmoid'))
    model.compile( optimizer=optimizer, loss='binary_crossentropy', metrics=['acc'])
    return model


In [3]:
f = open("complaint.txt",encoding="latin1")
negative = f.readlines()
f.close()
f = open("nocomplaint.txt",encoding="latin1")
positive = f.readlines()
f.close()
a = pd.DataFrame(zip(list(positive)),np.ones(1000))
b = pd.DataFrame(zip(list(negative)),np.zeros(1000))
df = pd.concat([a,b])
df["sentiment"] = df.index.values
df.columns = ["text","sentiment"]
data =df.reset_index(drop= True)

Firstly, the RNN model wound be used. The Simple RNN, LSTM and Bidirectional models had been tried. Only the last two model performed aacceptable result. However, the val accuracy is only around 65%. This may because the lack of training data.

In [4]:
max_words = 10000
max_len = 30
words =  clean(data["text"].tolist())

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(words)
sequences = tokenizer.texts_to_sequences(words)
train = pad_sequences(sequences, maxlen=max_len)

X_train, X_test, y_train, y_test = train_test_split(train, data["sentiment"], test_size=0.2, random_state=23)


In [5]:
y_train= to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

In [6]:
model1 = Sequential()
model1.add(layers.Embedding(max_words, 25)) #The embedding layer
model1.add(layers.LSTM(20,dropout=0.5)) #Our LSTM layer
model1.add(layers.Dense(10,activation='softmax'))
model1.add(layers.Dense(5,activation='softmax'))
model1.add(layers.Dense(2,activation='softmax'))
model1.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint1 = ModelCheckpoint("best_model1.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model1.fit(X_train, y_train, epochs=70,validation_data=(X_test, y_test),callbacks=[checkpoint1])


Epoch 1/70
50/50 [==============================] - 3s 28ms/step - loss: 0.7194 - accuracy: 0.5071 - val_loss: 0.7101 - val_accuracy: 0.4650

Epoch 00001: val_accuracy improved from -inf to 0.46500, saving model to best_model1.hdf5
Epoch 2/70
50/50 [==============================] - 0s 7ms/step - loss: 0.6934 - accuracy: 0.5252 - val_loss: 0.7001 - val_accuracy: 0.4650

Epoch 00002: val_accuracy did not improve from 0.46500
Epoch 3/70
50/50 [==============================] - 0s 6ms/step - loss: 0.6935 - accuracy: 0.5107 - val_loss: 0.6922 - val_accuracy: 0.4650

Epoch 00003: val_accuracy did not improve from 0.46500
Epoch 4/70
50/50 [==============================] - 0s 6ms/step - loss: 0.6841 - accuracy: 0.5062 - val_loss: 0.6797 - val_accuracy: 0.4650

Epoch 00004: val_accuracy did not improve from 0.46500
Epoch 5/70
50/50 [==============================] - 0s 6ms/step - loss: 0.6687 - accuracy: 0.6055 - val_loss: 0.6841 - val_accuracy: 0.5525

Epoch 00005: val_accuracy improved from

50/50 [==============================] - 0s 6ms/step - loss: 0.0561 - accuracy: 0.9884 - val_loss: 1.0667 - val_accuracy: 0.6700

Epoch 00041: val_accuracy did not improve from 0.74000
Epoch 42/70
50/50 [==============================] - 0s 6ms/step - loss: 0.0539 - accuracy: 0.9876 - val_loss: 1.0577 - val_accuracy: 0.6750

Epoch 00042: val_accuracy did not improve from 0.74000
Epoch 43/70
50/50 [==============================] - 0s 6ms/step - loss: 0.0644 - accuracy: 0.9880 - val_loss: 1.1792 - val_accuracy: 0.6675

Epoch 00043: val_accuracy did not improve from 0.74000
Epoch 44/70
50/50 [==============================] - 0s 6ms/step - loss: 0.0456 - accuracy: 0.9928 - val_loss: 1.0691 - val_accuracy: 0.6900

Epoch 00044: val_accuracy did not improve from 0.74000
Epoch 45/70
50/50 [==============================] - 0s 6ms/step - loss: 0.0431 - accuracy: 0.9933 - val_loss: 1.1038 - val_accuracy: 0.6750

Epoch 00045: val_accuracy did not improve from 0.74000
Epoch 46/70
50/50 [========

In [7]:
x = np.around(model1.predict(X_test),decimals=0).argmax(axis=1)
accuracy_score(x,y_test.argmax(axis=1))

0.655

In [8]:
model2 = Sequential()
model2.add(layers.Embedding(max_words, 40, input_length=max_len))
model2.add(layers.Bidirectional(layers.LSTM(20,dropout=0.6)))
model2.add(layers.Dense(2,activation='softmax'))
model2.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
checkpoint2 = ModelCheckpoint("best_model2.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model2.fit(X_train, y_train, epochs=70,validation_data=(X_test, y_test),callbacks=[checkpoint2])

Epoch 1/70
50/50 [==============================] - 4s 25ms/step - loss: 0.6893 - accuracy: 0.5470 - val_loss: 0.6657 - val_accuracy: 0.6275

Epoch 00001: val_accuracy improved from -inf to 0.62750, saving model to best_model2.hdf5
Epoch 2/70
50/50 [==============================] - 0s 8ms/step - loss: 0.6205 - accuracy: 0.6821 - val_loss: 0.5451 - val_accuracy: 0.7325

Epoch 00002: val_accuracy improved from 0.62750 to 0.73250, saving model to best_model2.hdf5
Epoch 3/70
50/50 [==============================] - 0s 8ms/step - loss: 0.4435 - accuracy: 0.7890 - val_loss: 0.5474 - val_accuracy: 0.7475

Epoch 00003: val_accuracy improved from 0.73250 to 0.74750, saving model to best_model2.hdf5
Epoch 4/70
50/50 [==============================] - 0s 8ms/step - loss: 0.3512 - accuracy: 0.8542 - val_loss: 0.5499 - val_accuracy: 0.7425

Epoch 00004: val_accuracy did not improve from 0.74750
Epoch 5/70
50/50 [==============================] - 0s 8ms/step - loss: 0.2845 - accuracy: 0.8825 - val_

In [9]:
x = np.around(model2.predict(X_test),decimals=0).argmax(axis=1)
accuracy_score(x,y_test.argmax(axis=1))

0.665

Here the DNN and some traditional classfication model would be used using the term frequency matrix.Firstly the word list would be created from our whole training dataset and then used as variables for future training.

In [10]:
a = (pd.Series(words).apply(lambda z :" ".join([x for x in z.split() if x not in stopword]))).tolist()

wordlist = []
for x in a:
    for word in x.split():
        if word not in wordlist:
            wordlist.append(word)


train = pd.DataFrame(doc2vec(a,wordlist))    
X_train2, X_test2, y_train2, y_test2 = train_test_split(train, data["sentiment"], test_size=0.2, random_state=23)

In [11]:
model_ann = KerasClassifier(build_fn=create_model, epochs=20, batch_size=500)

layer=[0,1,2,3]
param_grid = dict(layer=layer)
activation = [ 'relu', 'tanh']
param_grid['layer_activation'] = activation

optimizer = ['RMSprop']
param_grid['optimizer'] = optimizer  
dense=[10,15,16,18,20]
param_grid['dense']=dense
param_grid['epochs'] =[60,70]


yy=to_categorical(y_train2,2)
grid = GridSearchCV(estimator=model_ann, param_grid=param_grid,cv=2)
grid_result = grid.fit(X_train2,yy)


Epoch 1/60
2/2 [==============================] - 0s 8ms/step - loss: 0.6938 - acc: 0.4877
Epoch 2/60
2/2 [==============================] - 0s 9ms/step - loss: 0.6811 - acc: 0.7490
Epoch 3/60
2/2 [==============================] - 0s 9ms/step - loss: 0.6711 - acc: 0.8388
Epoch 4/60
2/2 [==============================] - 0s 9ms/step - loss: 0.6616 - acc: 0.8878
Epoch 5/60
2/2 [==============================] - 0s 11ms/step - loss: 0.6522 - acc: 0.9222
Epoch 6/60
2/2 [==============================] - 0s 9ms/step - loss: 0.6421 - acc: 0.9338
Epoch 7/60
2/2 [==============================] - 0s 8ms/step - loss: 0.6330 - acc: 0.9457
Epoch 8/60
2/2 [==============================] - 0s 9ms/step - loss: 0.6249 - acc: 0.9462
Epoch 9/60
2/2 [==============================] - 0s 10ms/step - loss: 0.6158 - acc: 0.9587
Epoch 10/60
2/2 [==============================] - 0s 8ms/step - loss: 0.6071 - acc: 0.9553
Epoch 11/60
2/2 [==============================] - 0s 9ms/step - loss: 0.5979 - acc: 0.

2/2 [==============================] - 0s 8ms/step - loss: 0.2611 - acc: 0.9835
Epoch 58/60
2/2 [==============================] - 0s 7ms/step - loss: 0.2547 - acc: 0.9855
Epoch 59/60
2/2 [==============================] - 0s 8ms/step - loss: 0.2501 - acc: 0.9848
Epoch 60/60
2/2 [==============================] - 0s 7ms/step - loss: 0.5550 - acc: 0.7325
Epoch 1/60
2/2 [==============================] - 1s 8ms/step - loss: 0.6920 - acc: 0.5497
Epoch 2/60
2/2 [==============================] - 0s 8ms/step - loss: 0.6630 - acc: 0.7748
Epoch 3/60
2/2 [==============================] - 0s 8ms/step - loss: 0.6431 - acc: 0.8510
Epoch 4/60
2/2 [==============================] - 0s 8ms/step - loss: 0.6283 - acc: 0.8862
Epoch 5/60
2/2 [==============================] - 0s 9ms/step - loss: 0.6122 - acc: 0.9012
Epoch 6/60
2/2 [==============================] - 0s 8ms/step - loss: 0.5987 - acc: 0.9168
Epoch 7/60
2/2 [==============================] - 0s 9ms/step - loss: 0.5869 - acc: 0.9232
Epoch 8

2/2 [==============================] - 0s 10ms/step - loss: 0.3825 - acc: 0.9668
Epoch 36/60
2/2 [==============================] - 0s 8ms/step - loss: 0.3774 - acc: 0.9668
Epoch 37/60
2/2 [==============================] - 0s 7ms/step - loss: 0.3701 - acc: 0.9677
Epoch 38/60
2/2 [==============================] - 0s 10ms/step - loss: 0.3632 - acc: 0.9668
Epoch 39/60
2/2 [==============================] - 0s 9ms/step - loss: 0.3578 - acc: 0.9677
Epoch 40/60
2/2 [==============================] - 0s 9ms/step - loss: 0.3507 - acc: 0.9698
Epoch 41/60
2/2 [==============================] - 0s 8ms/step - loss: 0.3467 - acc: 0.9678
Epoch 42/60
2/2 [==============================] - 0s 9ms/step - loss: 0.3396 - acc: 0.9680
Epoch 43/60
2/2 [==============================] - 0s 9ms/step - loss: 0.3328 - acc: 0.9705
Epoch 44/60
2/2 [==============================] - 0s 7ms/step - loss: 0.3249 - acc: 0.9723
Epoch 45/60
2/2 [==============================] - 0s 9ms/step - loss: 0.3183 - acc: 0.973

2/2 [==============================] - 0s 7ms/step - loss: 0.6505 - acc: 0.7883
Epoch 6/60
2/2 [==============================] - 0s 7ms/step - loss: 0.6403 - acc: 0.8213
Epoch 7/60
2/2 [==============================] - 0s 7ms/step - loss: 0.6292 - acc: 0.8383
Epoch 8/60
2/2 [==============================] - 0s 7ms/step - loss: 0.6186 - acc: 0.8688
Epoch 9/60
2/2 [==============================] - 0s 7ms/step - loss: 0.6067 - acc: 0.8813
Epoch 10/60
2/2 [==============================] - 0s 8ms/step - loss: 0.5959 - acc: 0.8933
Epoch 11/60
2/2 [==============================] - 0s 7ms/step - loss: 0.5822 - acc: 0.9057
Epoch 12/60
2/2 [==============================] - 0s 7ms/step - loss: 0.5719 - acc: 0.9130
Epoch 13/60
2/2 [==============================] - 0s 7ms/step - loss: 0.5600 - acc: 0.9238
Epoch 14/60
2/2 [==============================] - 0s 8ms/step - loss: 0.5467 - acc: 0.9253
Epoch 15/60
2/2 [==============================] - 0s 7ms/step - loss: 0.5359 - acc: 0.9423
Epoc

2/2 [==============================] - 0s 8ms/step - loss: 0.2512 - acc: 0.9893
Epoch 45/60
2/2 [==============================] - 0s 8ms/step - loss: 0.2431 - acc: 0.9908
Epoch 46/60
2/2 [==============================] - 0s 8ms/step - loss: 0.2379 - acc: 0.9938
Epoch 47/60
2/2 [==============================] - 0s 8ms/step - loss: 0.2299 - acc: 0.9938
Epoch 48/60
2/2 [==============================] - 0s 9ms/step - loss: 0.2253 - acc: 0.9953
Epoch 49/60
2/2 [==============================] - 0s 8ms/step - loss: 0.2190 - acc: 0.9955
Epoch 50/60
2/2 [==============================] - 0s 9ms/step - loss: 0.2138 - acc: 0.9955
Epoch 51/60
2/2 [==============================] - 0s 9ms/step - loss: 0.2087 - acc: 0.9968
Epoch 52/60
2/2 [==============================] - 0s 8ms/step - loss: 0.2039 - acc: 0.9977
Epoch 53/60
2/2 [==============================] - 0s 8ms/step - loss: 0.1984 - acc: 0.9962
Epoch 54/60
2/2 [==============================] - 0s 9ms/step - loss: 0.1933 - acc: 0.9970


...

2/2 [==============================] - 0s 10ms/step - loss: 0.3636 - acc: 0.9868
Epoch 21/70
2/2 [==============================] - 0s 8ms/step - loss: 0.3463 - acc: 0.9877
Epoch 22/70
2/2 [==============================] - 0s 11ms/step - loss: 0.3290 - acc: 0.9890
Epoch 23/70
2/2 [==============================] - 0s 8ms/step - loss: 0.3149 - acc: 0.9885
Epoch 24/70
2/2 [==============================] - 0s 10ms/step - loss: 0.2990 - acc: 0.9878
Epoch 25/70
2/2 [==============================] - 0s 9ms/step - loss: 0.2822 - acc: 0.9902
Epoch 26/70
2/2 [==============================] - 0s 9ms/step - loss: 0.2690 - acc: 0.9915
Epoch 27/70
2/2 [==============================] - 0s 11ms/step - loss: 0.2547 - acc: 0.9908
Epoch 28/70
2/2 [==============================] - 0s 9ms/step - loss: 0.2409 - acc: 0.9915
Epoch 29/70
2/2 [==============================] - 0s 9ms/step - loss: 0.2296 - acc: 0.9908
Epoch 30/70
2/2 [==============================] - 0s 9ms/step - loss: 0.2170 - acc: 0.9

2/2 [==============================] - 0s 10ms/step - loss: 0.0890 - acc: 0.9945
Epoch 40/70
2/2 [==============================] - 0s 9ms/step - loss: 0.0844 - acc: 0.9937
Epoch 41/70
2/2 [==============================] - 0s 11ms/step - loss: 0.0813 - acc: 0.9930
Epoch 42/70
2/2 [==============================] - 0s 10ms/step - loss: 0.0749 - acc: 0.9945
Epoch 43/70
2/2 [==============================] - 0s 11ms/step - loss: 0.0725 - acc: 0.9945
Epoch 44/70
2/2 [==============================] - 0s 9ms/step - loss: 0.0686 - acc: 0.9938
Epoch 45/70
2/2 [==============================] - 0s 11ms/step - loss: 0.0664 - acc: 0.9955
Epoch 46/70
2/2 [==============================] - 0s 9ms/step - loss: 0.0639 - acc: 0.9962
Epoch 47/70
2/2 [==============================] - 0s 13ms/step - loss: 0.0594 - acc: 0.9962
Epoch 48/70
2/2 [==============================] - 0s 11ms/step - loss: 0.0568 - acc: 0.9947
Epoch 49/70
2/2 [==============================] - 0s 11ms/step - loss: 0.0543 - acc:

2/2 [==============================] - 0s 11ms/step - loss: 0.0434 - acc: 0.9962
Epoch 58/70
2/2 [==============================] - 0s 10ms/step - loss: 0.0393 - acc: 0.9962
Epoch 59/70
2/2 [==============================] - 0s 10ms/step - loss: 0.0363 - acc: 0.9968
Epoch 60/70
2/2 [==============================] - 0s 11ms/step - loss: 0.0343 - acc: 0.9953
Epoch 61/70
2/2 [==============================] - 0s 11ms/step - loss: 0.0325 - acc: 0.9955
Epoch 62/70
2/2 [==============================] - 0s 11ms/step - loss: 0.0295 - acc: 0.9962
Epoch 63/70
2/2 [==============================] - ETA: 0s - loss: 0.0292 - acc: 0.994 - 0s 12ms/step - loss: 0.0282 - acc: 0.9947
Epoch 64/70
2/2 [==============================] - 0s 12ms/step - loss: 0.0265 - acc: 0.9947
Epoch 65/70
2/2 [==============================] - 0s 10ms/step - loss: 0.0233 - acc: 0.9968
Epoch 66/70
2/2 [==============================] - 0s 12ms/step - loss: 0.0229 - acc: 0.9955
Epoch 67/70
2/2 [===========================

2/2 [==============================] - 0s 12ms/step - loss: 0.0062 - acc: 1.0000
Epoch 67/70
2/2 [==============================] - 0s 8ms/step - loss: 0.0060 - acc: 1.0000
Epoch 68/70
2/2 [==============================] - 0s 9ms/step - loss: 0.0056 - acc: 1.0000
Epoch 69/70
2/2 [==============================] - 0s 9ms/step - loss: 0.0054 - acc: 1.0000
Epoch 70/70
2/2 [==============================] - 0s 7ms/step - loss: 1.0037 - acc: 0.7138
Epoch 1/70
2/2 [==============================] - 1s 9ms/step - loss: 0.6922 - acc: 0.5218
Epoch 2/70
2/2 [==============================] - 0s 10ms/step - loss: 0.6698 - acc: 0.6680
Epoch 3/70
2/2 [==============================] - 0s 8ms/step - loss: 0.6423 - acc: 0.7962
Epoch 4/70
2/2 [==============================] - 0s 9ms/step - loss: 0.6107 - acc: 0.8823
Epoch 5/70
2/2 [==============================] - 0s 11ms/step - loss: 0.5786 - acc: 0.9135
Epoch 6/70
2/2 [==============================] - 0s 9ms/step - loss: 0.5453 - acc: 0.9333
Epo

2/2 [==============================] - 0s 7ms/step - loss: 0.9724 - acc: 0.6925
Epoch 1/70
4/4 [==============================] - 1s 9ms/step - loss: 0.6931 - acc: 0.5004
Epoch 2/70
4/4 [==============================] - 0s 9ms/step - loss: 0.6769 - acc: 0.6722
Epoch 3/70
4/4 [==============================] - 0s 9ms/step - loss: 0.6628 - acc: 0.7483
Epoch 4/70
4/4 [==============================] - 0s 9ms/step - loss: 0.6487 - acc: 0.7937
Epoch 5/70
4/4 [==============================] - 0s 9ms/step - loss: 0.6365 - acc: 0.8082
Epoch 6/70
4/4 [==============================] - 0s 9ms/step - loss: 0.6235 - acc: 0.8439
Epoch 7/70
4/4 [==============================] - 0s 9ms/step - loss: 0.6111 - acc: 0.8475
Epoch 8/70
4/4 [==============================] - 0s 9ms/step - loss: 0.5977 - acc: 0.8748
Epoch 9/70
4/4 [==============================] - 0s 10ms/step - loss: 0.5848 - acc: 0.8758
Epoch 10/70
4/4 [==============================] - 0s 9ms/step - loss: 0.5738 - acc: 0.8709
Epoch 11

In [12]:
x = grid.predict(X_test2)
accuracy_score(x,y_test2)

C:\Users\USER\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.7325

The deep NN were trained by gridsearch model. Here we create a model generate function and then put into gridsearch function to find the parameters with highest validation accuracy. Here we can see that the accuracy is around 72%, which is acceptable for this dataset.

In [13]:
para_log = [{'max_iter': [10000],
             'penalty' : ['none', 'l2'],
             'tol':[0.0001,0.0002]
            }]


grid_log = GridSearchCV(LogisticRegression( random_state=0), para_log,cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=0))


grid_log.fit(X_train2, y_train2)
best_Log_l2_model = grid_log.best_estimator_
best_Log_l2_model.score(X_test2,y_test2)

0.765

Also, the logistic regression was used for the prediction. Both panelty with l1 and l2 were used and we found that l2 would generate better predict result. Thus, we only us l2 in the gridsearch function. The result suggested that the accuracy is around 0.765 in validation set. 

In [14]:
para_svc = [{'C':[0.5, 1, 2],
             'kernel':('linear', 'rbf', 'sigmoid')
            }]
grid_svm = GridSearchCV( svm.SVC(probability=True, class_weight="balanced"), para_svc, cv=2, scoring="accuracy",n_jobs=-1, verbose=1) 
grid_svm.fit(X_train2, np.array(y_train2))
best = grid_svm.best_params_
best1 = grid_svm.best_score_
print(grid_svm.score(X_test2, np.array(y_test2)))

Fitting 2 folds for each of 9 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:  1.7min finished


0.7475


Lastely, the SVM function was conducted. The best val accuracy is around 0.75. Thus, combine with two RNN related models, we'll estimate the noncomplaint with all five models and only the text with all positive outcome we'll category it as noncomplaint sentence. 

After we finished the model training, we can see that for most of models, they would have around 70% accuracy only. Thus, we may want to emsemble the model together to predict whether the sentence was the complaint or not. Here we only pick up the sentence which all five models predict it as noncomplaint and then randomly pick up 500 positive and 500 negative to have quick glance about the prediction power of our models.

In [61]:
test = pd.read_csv("customertweets.csv",header=None)
test.columns = ["index","text"]
test =test.loc[[len(re.sub("[^@]",'',x))<2 for x in test["text"]]]
tet = clean(test["text"])
sequences = tokenizer.texts_to_sequences(tet)
XXX = pad_sequences(sequences, maxlen=max_len)




In [16]:
test1 = doc2vec(tet,wordlist)
x = grid.predict(test1)
x1 = best_Log_l2_model.predict(test1)
x2 = grid_svm.predict(test1)
x3 =np.around(model1.predict(XXX),decimals=0).argmax(axis=1)
x4 = np.around(model2.predict(XXX),decimals=0).argmax(axis=1)

C:\Users\USER\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [17]:
a = x ==1
b = x1==1
c = x2==1
#d = x3==1
#e = x4==1


In [ ]:
#s = [x for x in a if (x in b and x in c and x in d and x in e)]
#l = [ x for x in test["index"] if x in s ]

#d = test.loc[[x in  l for x in test["index"]]]
#pp= []
#sia = SentimentIntensityAnalyzer()
#sentences = d["text"].tolist()
#for sentence in sentences:
#    ss = sia.polarity_scores(sentence)
#    pp.append(ss["compound"])
#dd = pd.concat([d.reset_index(drop = True), pd.DataFrame(pp,columns=["rate"])],axis=1)
#df_final =dd.sort_values("rate", ascending=False)
#df_final.to_csv("temp.csv")

After looking at the text in predicted positive and negative with the rule based model score, some issues could be identified. Firstly, the false positive rate is around 50% from the sample. For false negative, it's only around 20%. Meanwhile, the rule based score might be a great indicater while the number is small enough. For instance, in our sample, the score under -7 would always be false positive category. Thus, we may consider setting up threshold using rule based model and then reduce those have higher probability to be complaint text.

Meanwhile, as more data were labeled during the process, we tried to imporve our models by including more training data set.However, after running the models, only rnn related models shown improvement on the original val accuracy( since we afraid the new data might be labeled in different standard, the original validation set were used rather than regenerate the validation set.

In [18]:
df2 = pd.read_csv("Positive.csv")
df2.columns = ["text","sentiment","rate"]
df1 = pd.read_csv("Negative.csv")
df1.columns = ["text","sentiment","rate"]
data2 = pd.concat([df2[["text","sentiment"]].reset_index(drop=True),df1[["text","sentiment"]].reset_index(drop=True)])


In [19]:
words2 = clean(data2["text"].tolist())

max_words = 10000
max_len = 30

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(words)
sequences = tokenizer.texts_to_sequences(words2)
tweets = pad_sequences(sequences, maxlen=max_len)
y = data2["sentiment"]
X_train_f, X_test, y_train_f, y_test = train_test_split(tweets, y, test_size=0.2, random_state=23)

sequences = tokenizer.texts_to_sequences(words)
ts = pad_sequences(sequences, maxlen=max_len)
y_2 = data["sentiment"]
X_train_s, X_test,y_train_s, y_test = train_test_split(ts, y_2, test_size=0.2, random_state=23)



In [20]:
X_train = np.array(pd.concat([pd.DataFrame(X_train_f),pd.DataFrame(X_train_s)]))
y_train = pd.concat([pd.DataFrame(y_train_f),pd.DataFrame(y_train_s)])
y_train= to_categorical(y_train, 2)
y_test= to_categorical(y_test, 2)



In [21]:
model1n = Sequential()
model1n.add(layers.Embedding(max_words, 25)) #The embedding layer
model1n.add(layers.LSTM(20,dropout=0.5)) #Our LSTM layer
model1n.add(layers.Dense(10,activation='softmax'))
model1n.add(layers.Dense(5,activation='softmax'))
model1n.add(layers.Dense(2,activation='softmax'))
model1n.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint1 = ModelCheckpoint("best_model1.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model1n.fit(X_train, y_train, epochs=70,validation_data=(X_test, y_test),callbacks=[checkpoint1])


Epoch 1/70
76/76 [==============================] - 4s 15ms/step - loss: 0.7081 - accuracy: 0.4491 - val_loss: 0.6973 - val_accuracy: 0.4650

Epoch 00001: val_accuracy improved from -inf to 0.46500, saving model to best_model1.hdf5
Epoch 2/70
76/76 [==============================] - 1s 8ms/step - loss: 0.6858 - accuracy: 0.5728 - val_loss: 0.6923 - val_accuracy: 0.4650

Epoch 00002: val_accuracy did not improve from 0.46500
Epoch 3/70
76/76 [==============================] - 1s 9ms/step - loss: 0.6745 - accuracy: 0.5563 - val_loss: 0.6773 - val_accuracy: 0.4650

Epoch 00003: val_accuracy did not improve from 0.46500
Epoch 4/70
76/76 [==============================] - 1s 9ms/step - loss: 0.6510 - accuracy: 0.5722 - val_loss: 0.6654 - val_accuracy: 0.5575

Epoch 00004: val_accuracy improved from 0.46500 to 0.55750, saving model to best_model1.hdf5
Epoch 5/70
76/76 [==============================] - 1s 9ms/step - loss: 0.6320 - accuracy: 0.7246 - val_loss: 0.6428 - val_accuracy: 0.6900

E


Epoch 00036: val_accuracy did not improve from 0.74750
Epoch 37/70
76/76 [==============================] - 1s 9ms/step - loss: 0.0848 - accuracy: 0.9729 - val_loss: 0.9822 - val_accuracy: 0.6875

Epoch 00037: val_accuracy did not improve from 0.74750
Epoch 38/70
76/76 [==============================] - 1s 9ms/step - loss: 0.0762 - accuracy: 0.9810 - val_loss: 1.0282 - val_accuracy: 0.6800

Epoch 00038: val_accuracy did not improve from 0.74750
Epoch 39/70
76/76 [==============================] - 1s 9ms/step - loss: 0.0680 - accuracy: 0.9821 - val_loss: 1.0152 - val_accuracy: 0.6825

Epoch 00039: val_accuracy did not improve from 0.74750
Epoch 40/70
76/76 [==============================] - 1s 10ms/step - loss: 0.0733 - accuracy: 0.9809 - val_loss: 1.0140 - val_accuracy: 0.6825

Epoch 00040: val_accuracy did not improve from 0.74750
Epoch 41/70
76/76 [==============================] - 1s 10ms/step - loss: 0.0671 - accuracy: 0.9849 - val_loss: 1.0232 - val_accuracy: 0.6800

Epoch 00041:

In [23]:
model2n = Sequential()
model2n.add(layers.Embedding(max_words, 40, input_length=max_len))
model2n.add(layers.Bidirectional(layers.LSTM(20,dropout=0.6)))
model2n.add(layers.Dense(2,activation='softmax'))
model2n.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
checkpoint2 = ModelCheckpoint("best_model2.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model2n.fit(X_train, y_train, epochs=70,validation_data=(X_test, y_test),callbacks=[checkpoint2])


Epoch 1/70
76/76 [==============================] - 5s 22ms/step - loss: 0.6859 - accuracy: 0.5323 - val_loss: 0.6763 - val_accuracy: 0.4875

Epoch 00001: val_accuracy improved from -inf to 0.48750, saving model to best_model2.hdf5
Epoch 2/70
76/76 [==============================] - 1s 11ms/step - loss: 0.5830 - accuracy: 0.6533 - val_loss: 0.5290 - val_accuracy: 0.7525

Epoch 00002: val_accuracy improved from 0.48750 to 0.75250, saving model to best_model2.hdf5
Epoch 3/70
76/76 [==============================] - 1s 12ms/step - loss: 0.4186 - accuracy: 0.8064 - val_loss: 0.5353 - val_accuracy: 0.7575

Epoch 00003: val_accuracy improved from 0.75250 to 0.75750, saving model to best_model2.hdf5
Epoch 4/70
76/76 [==============================] - 1s 12ms/step - loss: 0.3248 - accuracy: 0.8628 - val_loss: 0.5588 - val_accuracy: 0.7025

Epoch 00004: val_accuracy did not improve from 0.75750
Epoch 5/70
76/76 [==============================] - 1s 12ms/step - loss: 0.2776 - accuracy: 0.8853 - 

In [31]:
X = np.around(model1n.predict(X_test),decimals=0).argmax(axis=1)
accuracy_score(X,y_test.argmax(axis=1))

0.6875

In [32]:
X = np.around(model2n.predict(X_test),decimals=0).argmax(axis=1)
accuracy_score(X,y_test.argmax(axis=1))

0.67

Here we can see roughly 0.02 improvement on val accuracy. THis may also indicate that the model still has large potential to improve.

After trying different threshold in our training dataset, we decided to use -0.5 as thershold for rule based model. If we observe that the score is below -0.5 then we classfy it as complaint

In [26]:
sentences = clean(data["text"].iloc[:1000,])
pp= []
sia = SentimentIntensityAnalyzer()
for sentence in sentences:
    ss = sia.polarity_scores(sentence)
    pp.append(ss["compound"])
len([x for x in  pp if x>-0.5])

896

In [27]:
score= []
sia = SentimentIntensityAnalyzer()
for sentence in tet:
    ss = sia.polarity_scores(sentence)
    score.append(ss["compound"])
raw = pd.DataFrame([tet,score],index= ["text",'score']).T
tet =raw["text"].loc[raw["score"]>-0.5]

#sequences = tokenizer.texts_to_sequences(tet)
#XXX = pad_sequences(sequences, maxlen=max_len)



After getting the update models and the test set, we can predict our final noncomplaint dataset. Different from last time, we would predict the noncomplaint text if the majority models suggest that the text is noncomplaint.

In [62]:
x3 =np.around(model1n.predict(XXX),decimals=0).argmax(axis=1)
x4 = np.around(model2n.predict(XXX),decimals=0).argmax(axis=1)

In [63]:
d = x3==1
e = x4==1



In [65]:
final =np.array(pd.concat([pd.DataFrame(a),pd.DataFrame(b),pd.DataFrame(c),pd.DataFrame(d),pd.DataFrame(e)],axis=1))

In [67]:
final.sum(1)

array([0, 0, 5, ..., 0, 1, 0])

In [136]:
extra = data2["text"].loc[data2["sentiment"]==1]
final_df = test.loc[final.sum(1)>2]
final2=test.loc[[x in extra.to_list() for x in test["text"].tolist()]]

In [141]:
final_df =pd.concat([final_df,final2])
final_df =final_df.drop_duplicates()

In [142]:
final_df.to_csv("project.csv")

In [3]:
df = pd.read_csv("project.csv")

In [4]:
pp= []
sia = SentimentIntensityAnalyzer()
sentences = df["text"].tolist()
for sentence in sentences:
    ss = sia.polarity_scores(sentence)
    pp.append(ss["compound"])

In [7]:
output = df[["index","text"]].loc[[x> -0.5 for x in pp]]
output.to_csv("Wei_ChenLee.csv")